In [6]:
from PyPDF2 import PdfReader

In [7]:
def read_pdf(file_path):
    with open(file_path, 'rb') as file:
        pdf_reader = PdfReader(file)
        num_pages = len(pdf_reader.pages)

        text = ''
        for page in pdf_reader.pages:
            text += page.extract_text()
        
        return text

In [8]:
book1_path = 'Main_project\\Noise a flaw in human judgement.pdf'
book2_path = 'ścieżka/do/książki2.pdf'

book1_text = read_pdf(book1_path)
#book2_text = read_pdf(book2_path)